In [271]:
import pytesseract
import cv2
from PIL import Image
from matplotlib import pyplot as plt

In [272]:
def display(img):
    cv2.imshow("img",img)
    cv2.waitKey(0)

In [273]:
img=cv2.imread("passport/8.PNG")
display(img)
max_h,max_w=img.shape[:2]

In [274]:
def rotateImage(image,skew_angle):
    r=0
    if (skew_angle > 45):
        r=-(90-skew_angle)
    else:
        r=skew_angle
    rows,cols=image.shape[:2]
    M1=cv2.getRotationMatrix2D(((cols-1)/2,(rows-1)/2),r,1)
    return (cv2.warpAffine(image,M1,(cols,rows)))

In [275]:
def contourize(image):
    img=image.copy()
    inverted=cv2.bitwise_not(img)
    gray=cv2.cvtColor(inverted,cv2.COLOR_BGR2GRAY)
    blurred=cv2.GaussianBlur(gray,(5,5),0)
    contours=[]
    t=0
    dilate=0
    while t < 256:
        _,thresh=cv2.threshold(blurred,t,255,cv2.THRESH_BINARY)
        temp_dilate=cv2.dilate(thresh,cv2.getStructuringElement(cv2.MORPH_RECT,(5,1)),iterations=3)
        temp_contours,_=cv2.findContours(temp_dilate,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        if len(temp_contours)>len(contours):
            contours=temp_contours
            thresh_value=t
            dilate=temp_dilate
        t+=1    
    
    return (contours,dilate) 

In [276]:
def deskew(Id):
    img=Id.copy()
    contours,dilate=contourize(img)
    skew_angles=[]
    mrz=[]
    for c in contours:
        (x , y , w , h)=cv2.boundingRect(c)
        if w>0.5*max_w and y>0.3*max_h:
            rect=cv2.minAreaRect(c)
            mrz.append(c)
            skew_angles.append(rect[-1])
    Avg_skew_angle=sum(skew_angles)/len(skew_angles)
    deskewed_img=rotateImage(img,Avg_skew_angle)
    deskewed_dilated=rotateImage(dilate,Avg_skew_angle)
    display(deskewed_img)
    contours=cv2.findContours(deskewed_dilated,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[0]
    margin=7
    for c in contours:
        (x,y,w,h)=cv2.boundingRect(c)
        if w>0.4*max_w and y>0.6*max_h and h<0.2*max_h:
            x=x-margin 
            y=y-margin
            w=w+2*margin
            h=h+2*margin
            print(pytesseract.image_to_string(deskewed_img[y:y+h,x:x+w]))
            display(deskewed_img[y:y+h,x:x+w])     

In [277]:
deskew(img)

7600884096RUS7401053F2902201<<<<<<<

PERUSMURZ I TSKOVAC<ELENA<<<<cccc<

